## keras下载

In [ ]:
!pip install keras==2.1.6 -i https://pypi.douban.com/simple --user

keras中文官网：https://keras.io/zh/

## Keras简单建模

In [9]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Reshape

# 定义输入层
input = Input(shape=(784,))
# 嵌入层
embedding = Embedding(784, 200, input_length=1)(input)
embedding = Reshape((200,))(embedding)

# 定义各个连接层，包括两个全连接层，使用relu和softmax激活函数
layer1 = Dense(64, activation='relu')(embedding)
layer2 = Dense(64, activation='relu')(layer1)

# 定义输出层
out = Dense(10, activation='softmax')(layer2)

# 定义模型对象
model = Model(inputs=input, output=out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 784)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1, 200)            156800    
_________________________________________________________________
reshape_2 (Reshape)          (None, 200)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                12864     
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_18 (Dense)             (None, 10)                650       
Total params: 174,474
Trainable params: 174,474
Non-trainable params: 0
_________________________________________________________________


/Users/jiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


## Embedding

将正整数（索引值）转换为固定尺寸的稠密向量。 例如： [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

该层只能用作模型中的第一层。

```python
keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)
```

#### 参数
- input_dim: int > 0。词汇表大小， 即，最大整数 index + 1。
- output_dim: int >= 0。词向量的维度。
- embeddings_initializer: embeddings 矩阵的初始化方法 (详见 initializers)。
- embeddings_regularizer: embeddings matrix 的正则化方法 (详见 regularizer)。
- embeddings_constraint: embeddings matrix 的约束函数 (详见 constraints)。
- mask_zero: 是否把 0 看作为一个应该被遮蔽的特殊的 "padding" 值。 这对于可变长的 循环神经网络层 十分有用。 如果设定为 True，那么接下来的所有层都必须支持 masking，否则就会抛出异常。 如果 mask_zero 为 True，作为结果，索引 0 就不能被用于词汇表中 （input_dim 应该与 vocabulary + 1 大小相同）。
- input_length: 输入序列的长度，当它是固定的时。 如果你需要连接 Flatten 和 Dense 层，则这个参数是必须的 （没有它，dense 层的输出尺寸就无法计算）。

#### 输入尺寸
尺寸为 (batch_size, sequence_length) 的 2D 张量。

#### 输出尺寸
尺寸为 (batch_size, sequence_length, output_dim) 的 3D 张量。

In [19]:
from keras.models import Sequential
import numpy as np

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=64, input_length=10))
# 模型将输入一个大小为 (batch, input_length) 的整数矩阵。
# 输入中最大的整数（即词索引）不应该大于 999 （词汇表大小）
# 现在 model.output_shape == (None, 10, 64)，其中 None 是 batch 的维度。

input_array = np.random.randint(1000, size=(32, 10))
print(input_array.shape)

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape == (32, 10, 64))

(32, 10)
True


## Dense

```python
keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
```
就是你常用的的全连接层。

Dense 实现以下操作： output = activation(dot(input, kernel) + bias) 其中 activation 是按逐个元素计算的激活函数，kernel 是由网络层创建的权值矩阵，以及 bias 是其创建的偏置向量 (只在 use_bias 为 True 时才有用)。

注意: 如果该层的输入的秩大于2，那么它首先被展平然后 再计算与 kernel 的点乘。

### 参数
- units: 正整数，输出空间维度。
- activation: 激活函数 (详见 activations)。 若不指定，则不使用激活函数 (即，「线性」激活: a(x) = x)。
- use_bias: 布尔值，该层是否使用偏置向量。
- kernel_initializer: kernel 权值矩阵的初始化器 (详见 initializers)。
- bias_initializer: 偏置向量的初始化器 (see initializers).
- kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。
- bias_regularizer: 运用到偏置向的的正则化函数 (详见 regularizer)。
- activity_regularizer: 运用到层的输出的正则化函数 (它的 "activation")。 (详见 regularizer)。
- kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。
- bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

### 输入尺寸
nD 张量，尺寸: (batch_size, ..., input_dim)。 最常见的情况是一个尺寸为 (batch_size, input_dim) 的 2D 输入。

### 输出尺寸
nD 张量，尺寸: (batch_size, ..., units)。 例如，对于尺寸为 (batch_size, input_dim) 的 2D 输入， 输出的尺寸为 (batch_size, units)。

In [18]:
# 作为 Sequential 模型的第一层
model = Sequential()
model.add(Dense(32, input_shape=(16,)))
# 现在模型就会以尺寸为 (*, 16) 的数组作为输入，
# 其输出数组的尺寸为 (*, 32)

# 在第一层之后，你就不再需要指定输入的尺寸了：
model.add(Dense(32))

## Dot
```python
keras.layers.Dot(axes, normalize=False)
```
计算两个tensor中样本的张量乘积。例如，如果两个张量a和b的shape都为（batch_size, n），则输出为形如（batch_size,1）的张量，结果张量每个batch的数据都是a[i,:]和b[i,:]的矩阵（向量）点积。

### 参数
- axes: 整数或整数的tuple，执行乘法的轴。
- normalize: 布尔值，是否沿执行成绩的轴做L2规范化，如果设为True，那么乘积的输出是两个样本的余弦相似性。
- kwargs: 普通的Layer关键字参数